In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import matplotlib.pyplot as plt


In [ ]:
# 5 year estimate of health care insurance(2015-2019)
data=pd.read_csv('/Users/naima.abdirahman/Downloads/productDownload_2024-11-16T162204/ACSDT5Y2019.B27010-Data.csv')
# make the 1st row the columns
data.columns=data.iloc[0]
# drow the 1st row
data=data[1:]
# remove under 19 and deduct from the total

data['Estimate!!Total:']=data['Estimate!!Total:'].astype(int)
data['Estimate!!Total:!!Under 19 years:']=data['Estimate!!Total:!!Under 19 years:'].astype(int)

data['Estimate!!Total:']=data['Estimate!!Total:']-data['Estimate!!Total:!!Under 19 years:']

# Ensure column names are treated as strings
data.columns = data.columns.astype(str)
data = data.drop([col for col in data.columns if 'Under 19' in col], axis=1)
# remove anyone with 2 or more insuraces and adjust the total
morathen1 = [col for col in data.columns if col.endswith('!!With two or more types of health insurance coverage:') 
             and 'Margin' not in col]


data[morathen1] = data[morathen1].astype(int)
data['Estimate!!Total:'] = data['Estimate!!Total:'].astype(int)

# Subtract the sum of the selected columns from the total
data['Estimate!!Total:'] = data['Estimate!!Total:'] - data[morathen1].sum(axis=1)

# drop margin or errors and with two or more insurances 
data=data.drop([col for col in data.columns if 'Margin' in col or 'With two' in col], axis=1)


# Function to classify the column names based on insurance type
def classify_insurance_type(col_name):
    if 'employer-based health insurance' in col_name:
        return 'Employer_based'
    if 'direct-purchase health insurance' in col_name:
        return 'Direct_purchase'
    if 'With Medicare coverage only' in col_name:
        return 'Medicare'
    if 'With Medicaid/means-tested' in col_name:
        return 'Medicaid'
    if 'TRICARE' in col_name:
        return 'TRICARE'
    if 'VA Health Care' in col_name:
        return 'VA_HealthCare'
    else:
        return 'None'

# Function to sum relevant columns for a given insurance type
def sum_insurance_type(data, insurance_type):
    # Identify the columns corresponding to the given insurance type
    relevant_columns = [col for col in data.columns if classify_insurance_type(col) == insurance_type]
    

    data[relevant_columns] = data[relevant_columns].apply(pd.to_numeric, errors='coerce')
        
        # Sum the valid numeric values across rows
    data[insurance_type] = data[relevant_columns].sum(axis=1)
        
        # Replace NaN values with 0 (for cases where the sum is NaN)
    data[insurance_type] = data[insurance_type].fillna(0).astype(int)
    
    return data

# Example: Applying the function to your data for different insurance types
insurance_types = ['Employer_based', 'Direct_purchase', 'Medicare', 'Medicaid', 'TRICARE', 'VA_HealthCare']

# Loop through each insurance type and apply the function
for insurance_type in insurance_types:
    data = sum_insurance_type(data, insurance_type)


data['Private']=data[['Employer_based', 'Direct_purchase', 'TRICARE']].sum(axis=1)
data['Public']=data[['Medicare', 'Medicaid', 'VA_HealthCare']].sum(axis=1)
data['NoInsurance']=data['Estimate!!Total:']-(data[['Private', 'Public']].sum(axis=1))



data=data[['Geography','Employer_based', 'Direct_purchase', 'TRICARE', 'Medicare', 'Medicaid', 'VA_HealthCare', 
           'Estimate!!Total:', 'Private', 'Public', 'NoInsurance']]

#
data['Geography'] = data['Geography'].str.split('US').str[-1]
data.rename(columns={'Geography':'LocationName'}, inplace=True)

def calculate_insurance_rate(data, insurance_columns, total_column='Estimate!!Total:'):
    """
    Calculate the rate of each insurance type relative to the 'Total' column.
    
    Parameters:
    - data: The DataFrame containing the insurance data.
    - insurance_columns: List of insurance columns to calculate the rates for.
    - total_column: The name of the column representing the total population (default is 'Total').
    
    Returns:
    - data: The updated DataFrame with new columns for each insurance rate.
    """
    for insurance_col in insurance_columns:
        # Create new columns for insurance rates
        rate_col_name = f'{insurance_col}_Rate'
        data[rate_col_name] = data[insurance_col] / data[total_column]
    
    return data

# List of insurance columns to calculate rates for
insurance_columns = ['Employer_based', 'Direct_purchase', 'TRICARE', 'Medicare', 'Medicaid', 
                     'NoInsurance', 'VA_HealthCare', 'Private', 'Public']

# Apply the function to calculate the insurance rates
data = calculate_insurance_rate(data, insurance_columns)
data.drop_duplicates(subset='LocationName', inplace=True)

# Splitting geolocation into latitude and longitude
#data[['Latitude', 'Longitude']] = data['GeoLocation'].str.extract(r'\((.+), (.+)\)').astype(float)


In [ ]:
#read the cancer data(CDC) and merge it with the insurance data
cdc=pd.read_csv('Clean_CDC.csv')
cdc['LocationName'] = cdc['LocationName'].astype(int)
data['LocationName'] = data['LocationName'].astype(int)
                                                             # Now perform the merge
merged= pd.merge(cdc, data, on='LocationName')
merged[['Latitude', 'Longitude']] = merged['GeoLocation'].str.extract(r'\((.+), (.+)\)').astype(float)

merged=merged[['Latitude','Longitude','Employer_based_Rate', 'Direct_purchase_Rate', 'TRICARE_Rate', 'Medicare_Rate', 
           'Medicaid_Rate', 'VA_HealthCare_Rate', '5Yr_Average',
            'Private_Rate', 'Public_Rate', 'NoInsurance_Rate', 'LocationName']]

merged.drop_duplicates(subset='LocationName', inplace=True)
merged.rename(columns={'5Yr_Average':'Cancer_rate'}, inplace=True)
merged.to_csv('Final.csv')

In [ ]:
# Distrubtion of the different types
cols_to_check = ['Employer_based_Rate', 'Direct_purchase_Rate', 'TRICARE_Rate', 'Medicare_Rate', 
                 'Medicaid_Rate', 'VA_HealthCare_Rate']

# Filter out rows where any of the selected columns have values <= 0
filtered_data = merged[(merged[cols_to_check] > 0).all(axis=1)]

# Create a boxplot comparing multiple insurance rate columns
plt.figure(figsize=(20,6))
sns.boxplot(data=filtered_data[cols_to_check])

# Add title and labels
plt.title('Boxplot of Insurance Rates')
plt.ylabel('Insurance Rate')  # Label for y-axis

# Display the plot
plt.show()

In [ ]:
#check for the correlation between the predictors 
# Select only the predictor columns
predictor_cols = [col for col in final.columns if col not in ['Latitude', 'Longitude', 'Cancer_rate', 'LocationName', 
                                                             'Private_Rate', 'Public_Rate']]

# Calculate the correlation matrix for the predictor variables
correlation_matrix = final[predictor_cols].corr()

sns.clustermap(correlation_matrix )

In [ ]:
# #OLS model
##Check for linearity 

coltoplot = ['Employer_based_Rate', 'Direct_purchase_Rate', 'TRICARE_Rate', 'Medicare_Rate', 
             'Medicaid_Rate', 'VA_HealthCare_Rate', 'Private_Rate', 'Public_Rate', 'NoInsurance_Rate']

# Loop over each column in coltoplot
for col in coltoplot:
    # Filter out non-positive values in both the column and the '5Yr_Average' column
    filtered_data = merged[(merged[col] > 0) & (merged['5Yr_Average'] > 0)]
    
    # Create scatter plot with a line of best fit
    plt.figure(figsize=(8, 6))  # Optional: Set figure size for better readability
    sns.regplot(data=filtered_data, x=col, y='5Yr_Average', scatter_kws={'s': 50}, line_kws={'color': 'red'}, ci=None)  # Red line for regression line
    
    # Calculate the correlation coefficient
    correlation = np.corrcoef(filtered_data[col].dropna(), filtered_data['5Yr_Average'].dropna())[0, 1]
    
    # Add title and correlation coefficient to the plot
    plt.title(f'{col} vs Cancer Rate')
    
    # Display the correlation on the plot
    plt.text(0.05, 0.95, f'Correlation: {correlation:.2f}', transform=plt.gca().transAxes, fontsize=12,
             verticalalignment='top', horizontalalignment='left', color='red')
    
    # Display the plot
    plt.show()



In [ ]:
# Model
merged=merged.dropna() # drop Nas


# Define the predictors and target variable
predictors = ['Employer_based_Rate', 'Direct_purchase_Rate', 'TRICARE_Rate', 'Medicare_Rate', 
              'Medicaid_Rate', 'VA_HealthCare_Rate']

# Extract the predictors and target variable
X = merged[predictors]
y = merged['Cancer_rate']

# Add a constant (intercept) to the predictors matrix
X = sm.add_constant(X)

# Fit the OLS model
ols_model = sm.OLS(y, X).fit()

# Get the summary of the regression
print(ols_model.summary())


# Get the residuals from the OLS model
residuals = ols_model.resid
fitted_values = ols_model.fittedvalues

# 1. Normality of residuals - Histogram and Q-Q plot
# Histogram
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(residuals, kde=True, bins=20)
plt.title('Histogram of Residuals')


# 2. Homoscedasticity (Equal Variance) - Residuals vs. Fitted values
plt.figure(figsize=(6, 6))
plt.scatter(fitted_values, residuals, color='blue', edgecolors='black', alpha=0.7)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Fitted Values')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.show()




In [ ]:
# Interactive Plot 


# Fill NaN values with 0 (or use another value as needed)
#data[['Latitude', 'Longitude']] = data['GeoLocation'].str.extract(r'\((.+), (.+)\)').astype(float)
merged['Cancer_rate'] = merged['Cancer_rate'].fillna(0)

# Now plot
import plotly.express as px

fig3 = px.scatter_mapbox(
    merged,
    lat="Latitude",
    lon="Longitude",
    hover_name="LocationName",
    hover_data=['Cancer_rate', 'Employer_based_Rate',
                'Direct_purchase_Rate', 'TRICARE_Rate', 'Medicare_Rate',
                'Medicaid_Rate', 'NoInsurance_Rate', 'VA_HealthCare_Rate'],
    color="Cancer_rate",  # Color points based on cancer prevalence
    size="Cancer_rate",   # based on NoInsurance_Rate
    zoom=6,
    mapbox_style="carto-positron"  
)
fig3.write_html("cancer_rate_map.html") # save plot 

fig3.show()

In [ ]:
# Cross Validation 

# OLS MODEL

#  Split the data into training and testing sets (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 
X_train = sm.add_constant(X_train)  # Add constant to the training set
X_test = sm.add_constant(X_test)  # Add constant to the testing set

# Fit the OLS model on the training data
ols_model = sm.OLS(y_train, X_train).fit()

# Make predictions on the test data
y_pred = ols_model.predict(X_test)

# Calculate RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Output the model summary and RMSE
print(ols_model.summary())
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Get residuals
residuals2 = ols_model_sqrt.resid
fitted2 = ols_model_sqrt.fittedvalues

# Residual vs Fitted plot
plt.figure(figsize=(8, 6))
sns.scatterplot(x=fitted2, y=residuals2, color='red')
plt.axhline(0, color='black', linestyle='--')
plt.title('Residual vs Fitted Values')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.show()